In [51]:
import os
from functools import partial
from multiprocessing.pool import Pool

import cv2
import youtube_dl
import numpy as np

In [81]:
NumProcesses = os.cpu_count()

def GetFrames(url, skip_frames, InitialOffset = 0, ProcessNumber = 0):
	global NumProcesses
	# Get the video capture
	cap = cv2.VideoCapture(url)

	# Video frame number
	FrameCount = cap.get(cv2.CAP_PROP_FRAME_COUNT)

	FramesPerProcess = FrameCount // NumProcesses
	FrameOffset = (ProcessNumber * FramesPerProcess) + InitialOffset

	CurrentFrame = FrameOffset
	cap.set(cv2.CAP_PROP_POS_FRAMES, FramesPerProcess)
	while CurrentFrame < FramesPerProcess + FrameOffset:
		ret, frame = cap.read()
		if not ret:
			break
		filename ="./images/PATH\shot" + str(CurrentFrame) + ".png"
		cv2.imwrite(filename, frame)
		CurrentFrame += skip_frames  # Skip 300 frames i.e. 10 seconds for 30 fps
		cap.set(1, CurrentFrame)
	cap.release()



video_url = "https://s9.bigcdn.cc/pubs/6198b1762b8d49.69649886/360.mp4"  # The Youtube URL

print("Obtaining frames")
with Pool(NumProcesses) as pool:
	pool.map(partial(GetFrames, video_url, 240, 2160), range(NumProcesses))

Obtaining frames
